#### Code for NWchem-Qiskit Driver
##### 1. import package

In [28]:
import os 
import sys
import yaml
import os
from qiskit_nwchem_driver import nwchem_driver
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit.exceptions import QiskitError
from qiskit_algorithms import VQE
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit.primitives import StatevectorEstimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD, UCC
import numpy as np
from qiskit_nwchem_driver.nwchem2yaml import extract_fields

In [29]:
# data_dir = os.path.join("qe_files", "n2", "output")
# datafile = os.path.join(data_dir, "demo.out")
# data_file_yaml = os.path.join(data_dir, "demo.yaml")

data_dir = os.path.join("..","data","PW_LiH_data", "3x3_periodic", "NWChem",'1covo')
datafile = os.path.join(data_dir, "H1Li1-2.0.out")
data_file_yaml = os.path.join(data_dir, "H1Li1-.0.yaml")

##### 2. nwchem output to yaml

In [30]:
data = extract_fields(datafile)
with open(data_file_yaml, 'w') as f:
    f.write(yaml.dump(data, default_flow_style=False)) 

##### 3. data to Qiskit electron structure problem class

In [31]:
driver = nwchem_driver.NWchem_Driver(data_file_yaml)
es_problem = driver.run()
# hamiltonian  = es_problem.hamiltonian
# print(hamiltonian.second_q_op())

[[-0.52469001  0.12481963  0.          0.        ]
 [ 0.12481963 -0.15544814  0.          0.        ]
 [ 0.          0.         -0.52469001  0.12481963]
 [ 0.          0.          0.12481963 -0.15544814]]
[[[[ 0.          0.          0.          0.        ]
   [ 0.          0.10703234  0.          0.        ]
   [ 0.          0.          0.12416203 -0.06240962]
   [ 0.          0.         -0.06240962  0.10703234]]

  [[ 0.          0.          0.          0.        ]
   [ 0.05675186  0.          0.          0.        ]
   [ 0.          0.         -0.06240962  0.05675186]
   [ 0.          0.          0.05675186 -0.06020991]]

  [[ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]]

  [[ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.        ]
   [ 0.          0.          0.          0.    

##### 4. Mapping Hamiltonian to qubit

In [32]:
mapper = JordanWignerMapper()

##### 5. specify ansatz

In [33]:
ansatz = UCCSD(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            es_problem.num_spatial_orbitals,
            es_problem.num_particles,
            mapper,
        ),
    )

##### 6. Constructing VQE and check results 

In [34]:
vqe_solver = VQE(Estimator(), ansatz, SLSQP())
#vqe_solver = VQE(StatevectorEstimator(), ansatz, SLSQP())
# vqe_solver.initial_point = [0.0] * ansatz.num_parameters
vqe_solver.initial_point = np.random.rand(ansatz.num_parameters)
calc = GroundStateEigensolver(mapper, vqe_solver)
res = calc.solve(es_problem)
print(res)
print('> Total energy from HF (Hartree):{}'.format(driver.total_energy))

/tmp/ipykernel_936/1423445988.py:1: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_solver = VQE(Estimator(), ansatz, SLSQP())


=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -0.945379353401
  - computed part:      -0.945379353401
~ Nuclear repulsion energy (Hartree): 0.0657458505
> Total ground state energy (Hartree): -0.879633502901
> Total energy from HF (Hartree):-0.7353101142
